In [ ]:
import datetime
startTime = datetime.now()

In [1]:
dataFrame = spark.read.format('csv').option('header','true').option('inferSchema','true').option('mode','DROPMALFORMED')\
.load("file:///media/alessandro/storage/big_data-primoProgetto/dataset/historical_stock_prices.csv")

In [2]:
dataFrame.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- adj_close: double (nullable = true)
 |-- low: double (nullable = true)
 |-- high: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- date: timestamp (nullable = true)



In [3]:
dataFrame.show(2,truncate= True)

+------+----------------+----------------+----------------+-----+----------------+-------+-------------------+
|ticker|            open|           close|       adj_close|  low|            high| volume|               date|
+------+----------------+----------------+----------------+-----+----------------+-------+-------------------+
|   AHH|            11.5|11.5799999237061|8.49315452575684|11.25|11.6800003051758|4633900|2013-05-08 00:00:00|
|   AHH|11.6599998474121|11.5500001907349|8.47115135192871| 11.5|11.6599998474121| 275800|2013-05-09 00:00:00|
+------+----------------+----------------+----------------+-----+----------------+-------+-------------------+
only showing top 2 rows



In [23]:
dataFrame.count() 

20973889

In [4]:
df = dataFrame.filter((dataFrame.date < '2018-01-01') & (dataFrame.date > '1998-01-01'))

In [67]:
df.count()

15946764

In [5]:
df = df.select('ticker','close', 'volume', 'date')

In [6]:
df.show(2)

+------+----------------+-------+-------------------+
|ticker|           close| volume|               date|
+------+----------------+-------+-------------------+
|   AHH|11.5799999237061|4633900|2013-05-08 00:00:00|
|   AHH|11.5500001907349| 275800|2013-05-09 00:00:00|
+------+----------------+-------+-------------------+
only showing top 2 rows



In [16]:
from pyspark.sql import functions as F

df1 = df.groupBy('ticker').agg((100*(F.last(df.close)-F.first(df.close))/F.first(df.close)).alias('percentageIncrease'),
                               F.min(df.close).alias('min_close'), 
                               F.max(df.close).alias('max_close'), 
                               F.mean(df.volume).alias('avg_volume'))

In [25]:
final = df1.orderBy(df1['percentageIncrease'].desc())

In [26]:
final.show(10)

+------+------------------+-------------------+----------------+------------------+
|ticker|percentageIncrease|          min_close|       max_close|        avg_volume|
+------+------------------+-------------------+----------------+------------------+
|   SAB|3864097.3488797136|                5.0|        318800.0| 1505460.228708134|
|   PJT| 227899.9974668022|0.00999999977648258|10947.2001953125| 63190.23985890653|
|  ORGS|183339.50194409874|0.00313999992795289|19.7999992370605|3871.6779825412223|
|  MNST| 170454.6912056308| 0.0305979158729315|64.2300033569336|7489340.8464763425|
|   PUB|151399.99957159112|0.00899999961256981|           135.0|  35213.1862170088|
|   RMP| 144042.3316475156| 0.0299999993294477|78.5498504638672|108796.61352040817|
|   UVE| 136650.0049639491| 0.0199999995529652|36.3199996948242|223893.03836633664|
|   CCD|100250.00071711814| 0.0149999996647239|25.5499992370605| 112677.6008010681|
|   STG| 91639.13043478261|   2.94000005722046|         36600.0| 92005.65220

In [ ]:
print('Exec Time: ')
print(datetime.now() - startTime)